In [24]:
import flask
from flask import jsonify
from vgg16_hybrid_places_1365 import VGG16_Hubrid_1365
from keras.preprocessing import image
from PIL import Image
import io
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam
from places_utils import preprocess_input

In [5]:
app = flask.Flask(__name__)

In [43]:
# load a pre trained model for feature formation. 
def get_model():
    global model
    model = VGG16_Hubrid_1365(weights='places', include_top=False)
    print ('model loaded')

In [63]:
# reconstruct the classifier which is a three layer fully connected layer
def get_classifier():
    global clf 
    clf = Sequential()
    clf.add(Dense(512, activation='relu', input_shape=(7*7*512,)))
    clf.add(Dropout(0.5))
    clf.add(Dense(256, activation='relu'))
    clf.add(Dropout(0.5))
    clf.add(Dense(128, activation='relu'))
    clf.add(Dropout(0.5))
    clf.add(Dense(1, activation='sigmoid'))

    # loads the weights 
    clf.load_weights('weights.best_features_3fc.h5')
    clf.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
    
    print ('classifier is reconstructed and loaded with weights')

In [13]:
def get_features(img, img_size):
    x = img.resize(img_size)
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    features = model.predict(x)
    
    return features
    

In [26]:
@app.route('/predict', methods =['GET','POST'])
def predict():
    if flask.request.method == 'POST':
        if flask.request.files.get('image'):
            # read the image in PIL format
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # preprocess the image and prepare it for classification
            features = get_features(image, img_size=(224, 224))
            # now use this features to predict using the classifier
            features = features.reshape((1,7*7*512))
            pred = clf.predict(features) # get the index of the class with higest prob
            print (pred)
            return jsonify ({'prediction': pred.tolist()})

AssertionError: View function mapping is overwriting an existing endpoint function: predict

In [68]:
if __name__ =='__main__':
    # frist load the pre-trained model and the fully connected classifier
    get_model()
    get_classifier()
    app.run()

model loaded


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


classifier is reconstructed and loaded with weights
